# Testings and benchmarkings for univariate scan functions

We will be using the BXD data for testing purposes. In this notebook, we are testing the univariate scan functionalities, by performing the functions on the 108th BXD trait. 

## Preparation:

In [1]:
pwd()

"/Users/zifanyu/Documents/GitHub/BulkLMM.jl/test/notebooks"

In [2]:
cd("..")

In [3]:
include("BXDdata_for_test.jl"); ## Load BXD data used for testing:

In [4]:
include("../src/parallel_helpers.jl");

In [5]:
kinship_Gc = CSV.read("run-gemma/output/kinship.cXX.txt", DataFrame, delim = '\t', header = false) |> Matrix;

In [6]:
using Test
using Plots

In [8]:
pheno_y = reshape(pheno[:, 108], :, 1);
(y0, X0, lambda0) = transform_rotation(pheno_y, geno, kinship_Gc); # by default will add an intercept to geno
X0_intercept = reshape(X0[:, 1], :, 1);
X0_covar = X0[:, 2:end];

## Test simple scans:

In [10]:
@time test_alt = scan(pheno_y, geno, kinship_Gc; reml = false, assumption = "alt", method = "qr");

  1.971169 seconds (5.79 M allocations: 2.600 GiB, 16.00% gc time)


In [20]:
lods_alt = test_alt[4];

In [58]:
findall(lods_alt .< 0) # confirm that uncorrected solution contains invalid LOD scores

241-element Vector{Int64}:
  229
  230
  231
  232
  233
  234
  235
  236
  237
  238
  239
  240
  241
    ⋮
 6112
 6113
 6114
 6115
 6116
 6117
 6118
 6119
 6123
 6124
 7227
 7228

perform correction by taking $(var(y_{108}), 0.1)$

In [22]:
var(pheno_y)

1.6944289276209026

In [23]:
prior = [var(pheno_y), 1/10]

2-element Vector{Float64}:
 1.6944289276209026
 0.1

In [18]:
@time test_alt_corrected = scan(pheno_y, geno, kinship_Gc; prior_a = prior[1], prior_b = prior[2], reml = false, assumption = "alt", method = "qr");

  0.856707 seconds (2.63 M allocations: 1.180 GiB, 15.98% gc time)


In [24]:
lods_alt_corrected = test_alt_corrected[4];

In [25]:
findall(lods_alt_corrected .< 0)

Int64[]

In [29]:
hcat(lods_alt, lods_alt_corrected)

7321×2 Matrix{Float64}:
 0.0219153  0.0199953
 0.0219153  0.0199953
 0.0219153  0.0199953
 0.0219153  0.0199953
 0.0219153  0.0199953
 0.0219153  0.0199953
 0.0219153  0.0199953
 0.0219153  0.0199953
 0.0219153  0.0199953
 0.0219153  0.0199953
 0.0219153  0.0199953
 0.021912   0.0199922
 0.0567256  0.0417663
 ⋮          
 0.248678   0.263758
 0.248678   0.263758
 0.248678   0.263758
 0.0686881  0.122407
 0.124881   0.136544
 0.124881   0.136544
 0.249213   0.255818
 0.273645   0.288192
 0.271594   0.285219
 0.285893   0.309153
 0.229505   0.219606
 0.229505   0.219606

In [27]:
@benchmark scan(pheno_y, geno, kinship_Gc; reml = false, assumption = "alt", method = "qr")

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.745 s …   1.780 s  ┊ GC (min … max): 12.12% … 11.32%
 Time  (median):     1.751 s              ┊ GC (median):    12.08%
 Time  (mean ± σ):   1.759 s ± 18.314 ms  ┊ GC (mean ± σ):  11.86% ±  0.47%

  █        █                                              █  
  █▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.75 s         Histogram: frequency by time        1.78 s <

 Memory estimate: 2.60 GiB, allocs estimate: 5791909.

In [28]:
@benchmark scan(pheno_y, geno, kinship_Gc; prior_a = prior[1], prior_b = prior[2], reml = false, assumption = "alt", method = "qr")

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  807.245 ms … 843.856 ms  ┊ GC (min … max): 12.93% … 14.11%
 Time  (median):     822.348 ms               ┊ GC (median):    12.70%
 Time  (mean ± σ):   826.035 ms ±  12.374 ms  ┊ GC (mean ± σ):  12.85% ±  0.89%

  █                   ██   █                 █    █           █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█ ▁
  807 ms           Histogram: frequency by time          844 ms <

 Memory estimate: 1.18 GiB, allocs estimate: 2625680.

In [35]:
@benchmark scan(pheno_y, geno, kinship_Gc; prior_a = prior[1], prior_b = prior[2], reml = false, assumption = "alt", method = "cholesky")

BenchmarkTools.Trial: 10 samples with 1 evaluation.
 Range (min … max):  534.107 ms … 588.627 ms  ┊ GC (min … max): 14.44% … 17.93%
 Time  (median):     555.800 ms               ┊ GC (median):    15.49%
 Time  (mean ± σ):   556.603 ms ±  13.846 ms  ┊ GC (mean ± σ):  15.51% ±  1.24%

  █            █      █ █████      █                          █  
  █▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁█████▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  534 ms           Histogram: frequency by time          589 ms <

 Memory estimate: 935.14 MiB, allocs estimate: 2256464.

In [36]:
@benchmark scan(pheno_y, geno, kinship_Gc; prior_a = prior[1], prior_b = prior[2], reml = false, assumption = "null", method = "cholesky")

BenchmarkTools.Trial: 221 samples with 1 evaluation.
 Range (min … max):  19.601 ms … 30.492 ms  ┊ GC (min … max):  0.00% … 15.63%
 Time  (median):     20.799 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   22.608 ms ±  2.669 ms  ┊ GC (mean ± σ):  10.01% ±  9.77%

   ▁█▂▁ ▂                                                      
  ▅████▇██▃▃▃▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▄▄▇█▆██▄▆▅▃▄▃▃▄▃▃▃▃▁▁▃▁▁▃▃▃▃▁▁▄ ▃
  19.6 ms         Histogram: frequency by time        28.6 ms <

 Memory estimate: 33.18 MiB, allocs estimate: 73514.

## Test scan with permutations:

## Exploring the batching idea:

In [48]:
function get_hsqs_from_null(Y::Array{Float64, 2}, K::Array{Float64, 2}; reml::Bool = false)

    (n, m) = size(Y);
    intercept = ones(n, 1);
    (Y0, intercept0, lambda0) = transform_rotation(Y, intercept, K; addIntercept = false);
    
    hsqs_from_null = Array{Float64, 1}(undef, m);
    
    prior = zeros(2);
    prior[2] = 1/100;
    
    for i in 1:m
        
        y0_i = reshape(Y0[:, i], :, 1);
        prior[1] = var(y0_i);
        
        out00 = fitlmm(y0_i, intercept0, lambda0, prior; reml = reml, method = "cholesky");
        hsqs_from_null[i] = out00.h2;
        
    end
    
    return hsqs_from_null;
    
end

get_hsqs_from_null (generic function with 1 method)

In [49]:
@time begin
    
    hsqs_from_null = get_hsqs_from_null(pheno, kinship_Gc; reml = false);
        
end

  8.891632 seconds (39.87 M allocations: 14.567 GiB, 15.86% gc time)


35556-element Vector{Float64}:
 5.252307989014744e-16
 3.0747546525840177e-15
 2.34890354044044e-15
 8.97201316153285e-16
 2.224913368023175e-15
 3.800605764727595e-15
 1.6536700148587646e-14
 3.6414645336939175e-14
 2.34890354044044e-15
 2.224913368023175e-15
 8.074343304670402e-15
 6.149509305168035e-15
 2.34890354044044e-15
 ⋮
 2.34890354044044e-15
 1.3750720834623256e-15
 0.1643203949220463
 0.07235333505058612
 8.97201316153285e-16
 2.466082349675042e-14
 4.06322116417833e-15
 2.73281150746183e-15
 6.400426481014958e-15
 0.06050234841340436
 6.149509305168035e-15
 1.375072083462324e-15

In [52]:
function get_hsqs_from_null_not_correct(Y::Array{Float64, 2}, K::Array{Float64, 2}; reml::Bool = false)

    (n, m) = size(Y);
    intercept = ones(n, 1);
    (Y0, intercept0, lambda0) = transform_rotation(Y, intercept, K; addIntercept = false);
    
    hsqs_from_null = Array{Float64, 1}(undef, m);
    prior = zeros(2);
    
    for i in 1:m
        
        y0_i = reshape(Y0[:, i], :, 1);
        
        out00 = fitlmm(y0_i, intercept0, lambda0, prior; reml = reml, method = "cholesky");
        hsqs_from_null[i] = out00.h2;
        
    end
    
    return hsqs_from_null;
    
end

get_hsqs_from_null_not_correct (generic function with 1 method)

In [53]:
@time begin
    
    hsqs_from_null_nc = get_hsqs_from_null_not_correct(pheno, kinship_Gc; reml = false);
        
end

  5.641205 seconds (25.06 M allocations: 9.147 GiB, 16.70% gc time)


35556-element Vector{Float64}:
 9.950115069895628e-15
 0.036528939421328205
 9.950115069895628e-15
 6.247563827463025e-15
 9.173598101430695e-15
 9.950115069895628e-15
 0.1305102674747561
 0.17410438338668452
 1.609962437506366e-14
 7.556752351802333e-15
 0.09142169755537538
 2.34890354044044e-15
 9.950115069895628e-15
 ⋮
 1.1034846708500515e-13
 2.6049739444959284e-14
 0.7218209389379234
 0.12763693428302544
 9.950115069895628e-15
 0.41810395059725675
 0.19611055191336954
 0.08632259430340487
 0.10927626211352637
 0.6094587891412765
 3.255168280819347e-13
 2.0235552928310703e-14

In [54]:
@benchmark get_hsqs_from_null(pheno, kinship_Gc; reml = false)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 8.851 s (15.77% GC) to evaluate,
 with a memory estimate of 14.57 GiB, over 39872075 allocations.

In [55]:
@benchmark get_hsqs_from_null_not_correct(pheno, kinship_Gc; reml = false)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 5.574 s (15.79% GC) to evaluate,
 with a memory estimate of 9.15 GiB, over 25059875 allocations.

In [57]:
maximum(hsqs_from_null)

0.9509585539087261

In [56]:
maximum(hsqs_from_null_nc)

0.999999984947842